In [79]:
using DifferentialEquations
using ADCME
using PyCall

In [7]:
function compute_cpHS(a, R, T)
    m = size(a)[1]
    cpHS = zeros(m, 3)
    for i = 1:m
        if (T > a[i,1])
            cpHS[i,1] = (a[i,2] + a[i,3] * T + a[i,4] * T^2 + a[i,5] * T^3 + a[i,6] * T^4) * R
            cpHS[i,2] = (a[i,2] + a[i,3] * T / 2 + a[i,4] * T^2 /3 + a[i,5] * T^3 /4 + a[i,6] * T^4 /5 + a[i,7]/T) * R * T
            cpHS[i,3] = (a[i,2] * log(T) + a[i,3] * T + a[i,4] * T^2 /2 + a[i,5] * T^3 /3 + a[i,6] * T^4 / 4 + a[i,8]) * R
        else
            cpHS[i,1] = (a[i,9] + a[i,10] * T + a[i,11] * T^2 + a[i,12] * T^3 + a[i,13] * T^4) * R
            cpHS[i,2] = (a[i,9] + a[i,10] * T / 2 + a[i,11] * T^2 /3 + a[i,12] * T^3 /4 + a[i,13] * T^4 /5 + a[i,14]/T) * R * T
            cpHS[i,3] = (a[i,9] * log(T) + a[i,10] * T + a[i,11] * T^2 /2 + a[i,12] * T^3 /3 + a[i,13] * T^4 / 4 + a[i,15]) * R        
        end
    end
    return cpHS
end

compute_cpHS (generic function with 1 method)

In [8]:
#### Read data
py"""
import numpy as np
"""

In [64]:
V = 1  # Volume
m = 0.21703355349342215 # Total mass
NASA_coeffs = py"np.load"("data/NASA_coeffs.npy")
W = py"np.load"("data/molecular_weights.npy") # Molar weight
ν1_order = py"np.load"("data/reaction_orders.npy") # Forward molar stoichiometric coefficients
ν1 = py"np.load"("data/reactants_stoich_coeffs.npy") # Forward molar stoichiometric coefficients
ν2 = py"np.load"("data/product_stoich_coeffs.npy") # Backward model stoichiometric coefficients
N = size(ν1)[1]  # Number of Species
M = size(ν1)[2]  # Number of Reactions
ν = ν2 - ν1  
pa = 100000 # 1 bar
R = 8314.46261815324 # Gas constant in kmol
### Constants: Combustion Chamber Level
min_dot = 0 # Rate at which mass enters the chamber
mout_dot = 0 # Rate at which mass leaves the chamber
Yin = ones(N) # Mass fraction of species entering the chamber
Yout = ones(N) # Mass fraction of species leaving the chamber
Qdot = 0 # Heating source
m_dot = min_dot - mout_dot
hin = 1 # Enthalpy of input
### Unknowns
T = 1200.0 # Temperature
Y = [0.5; 0; 0.5; 0; 0] # Mass fractions
### Quantities that depend on the Unknowns
ρ = m / V # density
Af = [3.47850e+08] # preexponential constant Afj
β = [0] # Temperature exponent
E = [83680000.0] # Activation energy for the reactions in kJ (divide the value from Cantera by 1000)
Q = ones(M) # Individual progress rates
Kf = ones(M) # Forward reaction coefficients
Kr = zeros(M) # Reverse reaction coefficients
##### Computing ω_dot 
cpHS = compute_cpHS(NASA_coeffs, R, T)
cp = cpHS[:,1]
h = cpHS[:,2]
s = cpHS[:,3]
cvk = cp .- R
ΔS = ν' * s  # Entropy change for reaction j
ΔH = ν' * h # Entahlpy change for reaction j
X = ρ * Y ./ W # Concentration
cv = sum(cvk ./ W .* Y) # Mass heat capacities
u = h ./ W - R ./ W .* T   # Internal energy for species
p = sum(X) * R * T # pressure
Kf = Af .* (T .^ β) .* exp.(-E ./ (R * T))
Kr = Kf ./ (((pa/(R * T)) .^ sum(ν, dims=1)' .* exp.(ΔS ./ R - ΔH ./ (R * T))))
Q = Kf .* prod(X .^ ν1_order, dims=1)' .- Kr .* prod(X .^ ν2, dims=1)'
ω_dot = W .* sum(ν .* Q', dims=2)
###### Species Conservation
mgen_dot = V .* ω_dot 
Y_dot = (1 / m) .* min_dot .* (Yin .- Y) - mout_dot .* Y .+ mgen_dot 
###### Energy Conservation
T_dot = 1 / (m * cv) * (-Qdot + min_dot * (hin - sum(u .* Yin)) - p * V / m * mout_dot - sum(mgen_dot .* u))

4.268052226065232e7

In [94]:
function f(dTY, TY, p, t)
    T = TY[1]
    Y = TY[2:end]
    cpHS = compute_cpHS(NASA_coeffs, R, T)
    cp = cpHS[:,1]
    h = cpHS[:,2]
    s = cpHS[:,3]
    cvk = cp .- R
    ΔS = ν' * s  # Entropy change for reaction j
    ΔH = ν' * h # Entahlpy change for reaction j
    X = ρ * Y ./ W # Concentration
    cv = sum(cvk ./ W .* Y) # Mass heat capacities
    u = h ./ W - R ./ W .* T   # Internal energy for species
    p = sum(X) * R * T # pressure
    Kf = Af .* (T .^ β) .* exp.(-E ./ (R * T))
    Kr = Kf ./ (((pa/(R * T)) .^ sum(ν, dims=1)' .* exp.(ΔS ./ R - ΔH ./ (R * T))))
    Q = Kf .* prod(X .^ ν1_order, dims=1)' .- Kr .* prod(X .^ ν2, dims=1)'
    ω_dot = W .* sum(ν .* Q', dims=2)
    ###### Species Conservation
    mgen_dot = V .* ω_dot 
    Y_dot = (1 / m) .* min_dot .* (Yin .- Y) - mout_dot .* Y .+ mgen_dot 
    ###### Energy Conservation
    T_dot = 1 / (m * cv) * (-Qdot + min_dot * (hin - sum(u .* Yin)) - p * V / m * mout_dot - sum(mgen_dot .* u))
    dTY = [T_dot; Y_dot]
    return dTY
end

function f(TY)
    T = TY[1]
    Y = TY[2:end]
    cpHS = compute_cpHS(NASA_coeffs, R, T)
    cp = cpHS[:,1]
    h = cpHS[:,2]
    s = cpHS[:,3]
    cvk = cp .- R
    ΔS = ν' * s  # Entropy change for reaction j
    ΔH = ν' * h # Entahlpy change for reaction j
    X = ρ * Y ./ W # Concentration
    cv = sum(cvk ./ W .* Y) # Mass heat capacities
    u = h ./ W - R ./ W .* T   # Internal energy for species
    p = sum(X) * R * T # pressure
    Kf = Af .* (T .^ β) .* exp.(-E ./ (R * T))
    Kr = Kf ./ (((pa/(R * T)) .^ sum(ν, dims=1)' .* exp.(ΔS ./ R - ΔH ./ (R * T))))
    Q = Kf .* prod(X .^ ν1_order, dims=1)' .- Kr .* prod(X .^ ν2, dims=1)'
    ω_dot = W .* sum(ν .* Q', dims=2)
    ###### Species Conservation
    mgen_dot = V .* ω_dot 
    Y_dot = (1 / m) .* min_dot .* (Yin .- Y) - mout_dot .* Y .+ mgen_dot 
    ###### Energy Conservation
    T_dot = 1 / (m * cv) * (-Qdot + min_dot * (hin - sum(u .* Yin)) - p * V / m * mout_dot - sum(mgen_dot .* u))
    return [T_dot; Y_dot]
end

f (generic function with 3 methods)

In [149]:
## RK4
dt = 1e-11
TY = [T;Y]
Tt = zeros(5000)
Pt = zeros(5000)
Yt = zeros(N,5000)
Y0 = [0.5; 0; 0.5; 0; 0] 
for i = 1:500000
    k1 = f(TY)
    k2 = f(TY + 0.5 * dt * k1)
    k3 = f(TY + 0.5 * dt * k2)
    k4 = f(TY + dt * k3)
    TY += 1/6 * dt * (k1 + 2 * k2 + 2 * k3 + k4)
    if (i % 100 == 0)
        Tt[Int(i / 100)] = TY[1]
        Y = TY[2:end]
        Yt[:,Int(i / 100)] = Y - Y0
        X = ρ * Y ./ W
        p = sum(X) * R * TY[1]
        Pt[Int(i / 100)] = p
    end
end


In [129]:
using Plots

┌ Info: Precompiling Plots [91a5bcdd-55d7-5caf-9e0b-520d859cae80]
└ @ Base loading.jl:1278


In [186]:
X = [i * 1e-9 for i = 1:5000]
Yt_cant = py"np.load"("data/mass_fraction.npy")
plot(X, Yt', label=["O2" "H20" "CH4" "CO2" "N2"])
xlabel!("Time")
ylabel!("Change in mass fractions")
savefig("Mass_fraction.pdf")

In [183]:
Yt_cant

5×5000 Array{Float64,2}:
 0.499991    0.499982    0.499972    …  0.270987  0.27075   0.270512
 5.18291e-6  1.03673e-5  1.55531e-5     0.128934  0.129068  0.129202
 0.499998    0.499995    0.499993       0.442592  0.442532  0.442472
 6.3307e-6   1.26632e-5  1.89975e-5     0.157487  0.157651  0.157814
 0.0         0.0         0.0            0.0       0.0       0.0

In [187]:
X = [i * 1e-9 for i = 1:5000]
Pt_cant = py"np.load"("data/pressure.npy")
plot(X, [Pt Pt_cant], label = ["Julia" "Cantera"])
xlabel!("Time")
ylabel!("Pressure")
savefig("Pressure.pdf")

In [188]:
plot(X, Tt)
Tt_cant = py"np.load"("data/temperature.npy")
plot(X, [Tt Tt_cant], label = ["Julia" "Cantera"])
xlabel!("Time")
ylabel!("Temperature")
savefig("Temperautre.pdf")

In [173]:
TY

6×1 Array{Float64,2}:
 2813.2377516141873
    0.4077206109618099
    0.051953167986047204
    0.47686763110979935
    0.06345858994232516
    0.0

In [ ]:
compute_cpHS1(NASA_coeffs[1,9:15], 8314.4621, T) 

In [ ]:
X

In [ ]:
V = 0.7693166084401741  # Volume
m = 1 # Total mass
N = 4  # Number of Species
M = 1  # Number of Reactions
W = [16.04276; 31.9988; 44.0098; 18.01528] # Molar weight
cvk = [1710.8, 658.57, 656.75, 1403.4]
ν1_order = [1; 0.5; 0; 0] # Forward molar stoichiometric coefficients
ν1 = [2; 1; 0; 0] # Forward molar stoichiometric coefficients
ν2 = [0; 0; 1; 2] # Backward model stoichiometric coefficients
forward_order = [1; 2; 0; 0] # Forward reaction order
reverse_order = [0; 0; 1; 2] # Reverse reaction order
ν = ν2 - ν1  
S0 = [186.3; 205.1; 213.7; 188.8]
H0 = [-74.8; 0; -393.5; -241.8]
ΔS = [-5163.4343755]  # Entropy change for reaction j
ΔH = [-8.0253912e+08] # Entahlpy change for reaction j
pa = 100000 # 1 bar
R = 8314.46261815324 # Gas constant in kmol
### Constants: Combustion Chamber Level
min_dot = 0 # Rate at which mass enters the chamber
mout_dot = 0 # Rate at which mass leaves the chamber
Yin = ones(N) # Mass fraction of species entering the chamber
Yout = ones(N) # Mass fraction of species leaving the chamber
Qdot = 0 # Heating source
m_dot = min_dot - mout_dot
hin = 1 # Enthalpy of input
### Unknowns
T = 300 # Temperature
Y = [0.25; 0.25; 0.25; 0.25] # Mass fractions
### Quantities that depend on the Unknowns
ρ = m / V # density
Af = [3.47850e+08] # preexponential constant Afj
β = [0] # Temperature exponent
E = [83680000.0] # Activation energy for the reactions in kJ (divide the value from Cantera by 1000)
Q = ones(M) # Individual progress rates
Kf = ones(M) # Forward reaction coefficients
Kr = zeros(M) # Reverse reaction coefficients

##### Computing ω_dot 
c_v = sum(cvk .* Y) # Mass heat capacities
X = ρ * Y ./ W # Concentration
u = X .* cvk .* W .* T   # Internal energy for species
p = sum(X) * R * T # pressure
Kf = Af .* (T .^ β) .* exp.(-E ./ (R * T))
Kr = Kf ./ (((pa/(R * T)) .^ sum(ν, dims=1)' .* exp.(ΔS ./ R - ΔH ./ (R * T))))
Q = Kf .* prod(X .^ ν1_order, dims=1)' .- Kr .* prod(X .^ ν2, dims=1)'
ω_dot = W .* sum(ν .* Q', dims=2)
###### Species Conservation
mgen_dot = V .* ω_dot .* W
Y_dot = (1 / m) .* min_dot .* (Yin .- Y) - mout_dot .* Y .+ mgen_dot 
###### Energy Conservation
T_dot = 1 / (m * c_v) * (-Qdot + min_dot * (hin - sum(u .* Yin)) - p * V / m * mout_dot - sum(mgen_dot .* u))
